In [167]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
from sklearn.utils import resample
from scipy.stats import zscore

# Load the dataset
df = pd.read_csv('winequality-red.csv', header = 0, delimiter=';')

# Splitting data by class
df_3 = df[df.quality ==3]
df_4 = df[df.quality ==4]
df_5 = df[df.quality ==5]
df_6 = df[df.quality ==6]
df_7 = df[df.quality ==7]
df_8 = df[df.quality ==8]

# Defining function that outputs a dataframe based on meeting the 3 standard deviation criteria 
def remove_outliers(df):
    z = np.abs(zscore(df.drop(['quality'], axis=1)))
    df_out = df[(z < 3).all(axis=1)]
    return df_out

# Applying the function to each dataframe
df_3_new = remove_outliers(df_3)
df_4_new = remove_outliers(df_4)
df_5_new = remove_outliers(df_5)
df_6_new = remove_outliers(df_6)
df_7_new = remove_outliers(df_7)
df_8_new = remove_outliers(df_8)

# Concatenating all new dataframes by row
df_new = pd.concat([df_3_new,df_4_new,df_5_new,df_6_new,df_7_new,df_8_new],axis=0)

# Prepare the data
X = df_new.drop('quality', axis=1)
y = df_new['quality']

# Spltting the data set 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [168]:
# Prints out the new data set with the new number of counts, min, max, mean and STD.
df_new.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000,1448.000000
mean,8.308840,0.524997,0.264448,2.386809,0.081497,15.163674,44.161602,0.996717,3.316133,0.641740,10.408943,5.636050
std,1.638422,0.172313,0.190651,0.863379,0.020699,9.332461,30.125723,0.001732,0.140308,0.129001,1.012177,0.810777
min,4.900000,0.120000,0.000000,1.200000,0.012000,1.000000,6.000000,0.990640,2.880000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.071000,7.000000,21.000000,0.995600,3.220000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,36.000000,0.996700,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.635000,0.420000,2.600000,0.089000,21.000000,59.000000,0.997800,3.400000,0.720000,11.000000,6.000000
max,13.700000,1.580000,0.760000,6.700000,0.267000,48.000000,155.000000,1.002600,3.740000,1.130000,14.000000,8.000000


In [153]:
from sklearn.metrics import classification_report

# Defining a pipeline with standard scaling, RFE feature selection using RFC, and RFC classification
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42,class_weight='balanced'))),
    ('classification', RandomForestClassifier(random_state=42,class_weight='balanced'))
])

# Defining a dictionary parameter grid to test out for the feature selection and classification parameters
param_grid = {
    'feature_selection__n_features_to_select': [5,6,7,8,9,10,11],
    'classification__n_estimators': [98,99,100,101,102],
    'classification__max_depth': [None],
    'classification__min_samples_split': [1,2,3,4,5,6],
    'classification__min_samples_leaf': [1,2,3,4],
    'classification__bootstrap': [True],
}

# Defining the grid search with the pipeline, parameter grid, and weighted scoring
grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtaining best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Obtaining prediction results
y_pred = grid_search.predict(X_test)

# Printing the classification report
print(classification_report(y_test, y_pred))


Best parameters: {'classification__bootstrap': True, 'classification__max_depth': None, 'classification__min_samples_leaf': 1, 'classification__min_samples_split': 4, 'classification__n_estimators': 102, 'feature_selection__n_features_to_select': 6}
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.50      0.10      0.17        10
           5       0.67      0.76      0.71       123
           6       0.63      0.64      0.63       116
           7       0.73      0.61      0.67        36
           8       0.00      0.00      0.00         3

    accuracy                           0.66       290
   macro avg       0.42      0.35      0.36       290
weighted avg       0.64      0.66      0.64       290



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [154]:
from sklearn.metrics import classification_report

# Defining a pipeline with standard scaling, RFE feature selection using RFC, and RFC classification
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42,class_weight='balanced'))),
    ('classification', RandomForestClassifier(random_state=42,class_weight='balanced'))
])

# Defining a dictionary parameter grid to test out for the feature selection and classification parameters. This is altered from above to see what happenes when we remove the option of all features
param_grid = {
    'feature_selection__n_features_to_select': [7,8,9,10],
    'classification__n_estimators': [98,99,100,101,102],
    'classification__max_depth': [None],
    'classification__min_samples_split': [4,5,6],
    'classification__min_samples_leaf': [1,2,3,4],
    'classification__bootstrap': [True],
}

grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Make predictions
y_pred = grid_search.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))


Best parameters: {'classification__bootstrap': True, 'classification__max_depth': None, 'classification__min_samples_leaf': 2, 'classification__min_samples_split': 4, 'classification__n_estimators': 99, 'feature_selection__n_features_to_select': 8}
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.25      0.10      0.14        10
           5       0.68      0.80      0.74       123
           6       0.68      0.64      0.66       116
           7       0.69      0.61      0.65        36
           8       0.00      0.00      0.00         3

    accuracy                           0.68       290
   macro avg       0.38      0.36      0.36       290
weighted avg       0.66      0.68      0.66       290



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [170]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
from sklearn.utils import resample
from scipy.stats import zscore

# Loading redwine quality data
df = pd.read_csv('winequality-red.csv', header = 0, delimiter=';')

# Splitting data by class
df_3 = df[df.quality ==3]
df_4 = df[df.quality ==4]
df_5 = df[df.quality ==5]
df_6 = df[df.quality ==6]
df_7 = df[df.quality ==7]
df_8 = df[df.quality ==8]

# Defining a function that removes outliers by IQR out side of the area between the 1st and 3rd quartile
def remove_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    # This looks for any value that is within the 1st and 3rd quartile, and negates the boolean value to return rows that are not within it
    new_df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return new_df

# Applying the function to each dataframe
df_3_new = remove_outliers(df_3)
df_4_new = remove_outliers(df_4)
df_5_new = remove_outliers(df_5)
df_6_new = remove_outliers(df_6)
df_7_new = remove_outliers(df_7)
df_8_new = remove_outliers(df_8)

# Concatenating all new dataframes by row
df_new = pd.concat([df_3_new,df_4_new,df_5_new,df_6_new,df_7_new,df_8_new],axis=0)

# Prepare the data
X = df_new.drop('quality', axis=1)
y = df_new['quality']

# Spltting the data set 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [171]:
df_new.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000,1166.000000
mean,8.207633,0.522316,0.254657,2.195369,0.078858,15.247856,44.070326,0.996645,3.319605,0.631955,10.314994,5.626072
std,1.452245,0.166501,0.181233,0.441612,0.013857,8.828036,29.413323,0.001541,0.131017,0.115711,0.944769,0.766782
min,5.200000,0.120000,0.000000,1.200000,0.038000,1.000000,6.000000,0.991910,2.940000,0.330000,8.700000,3.000000
25%,7.200000,0.390000,0.090000,1.900000,0.070000,8.000000,23.000000,0.995600,3.230000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.240000,2.100000,0.078000,14.000000,36.000000,0.996675,3.320000,0.610000,10.033333,6.000000
75%,9.000000,0.630000,0.400000,2.500000,0.087000,21.000000,58.000000,0.997600,3.400000,0.700000,11.000000,6.000000
max,13.300000,1.185000,0.750000,4.250000,0.123000,43.000000,153.000000,1.001000,3.690000,1.050000,13.600000,8.000000


In [157]:
from sklearn.metrics import classification_report

# Defining a pipeline with standard scaling, RFE feature selection using RFC, and RFC classification
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42,class_weight='balanced'))),
    ('classification', RandomForestClassifier(random_state=42,class_weight='balanced'))
])

# Defining a dictionary parameter grid to test out for the feature selection and classification parameters
param_grid = {
    'feature_selection__n_features_to_select': [5,6,7,8,9,10,11],
    'classification__n_estimators': [98,99,100,101,102],
    'classification__max_depth': [None],
    'classification__min_samples_split': [1,2,3,4,5,6],
    'classification__min_samples_leaf': [1,2,3,4],
    'classification__bootstrap': [True],
}

# Defining the grid search with the pipeline, parameter grid, and weighted scoring
grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtaining best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Obtaining prediction results
y_pred = grid_search.predict(X_test)

# Printing the classification report
print(classification_report(y_test, y_pred))


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'classification__bootstrap': True, 'classification__max_depth': None, 'classification__min_samples_leaf': 1, 'classification__min_samples_split': 4, 'classification__n_estimators': 99, 'feature_selection__n_features_to_select': 11}
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         7
           5       0.73      0.86      0.79       100
           6       0.72      0.72      0.72        98
           7       0.82      0.54      0.65        26
           8       1.00      0.50      0.67         2

    accuracy                           0.74       234
   macro avg       0.55      0.44      0.47       234
weighted avg       0.71      0.74      0.72       234



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [158]:
from sklearn.metrics import classification_report

# Defining a pipeline with standard scaling, RFE feature selection using RFC, and RFC classification
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42,class_weight='balanced'))),
    ('classification', RandomForestClassifier(random_state=42,class_weight='balanced'))
])

# Defining a dictionary parameter grid to test out for the feature selection and classification parameters. This is altered from above to see what happenes when we remove the option of all features
param_grid = {
    'feature_selection__n_features_to_select': [7,8,9,10],
    'classification__n_estimators': [98,99,100,101,102],
    'classification__max_depth': [None],
    'classification__min_samples_split': [4,5,6],
    'classification__min_samples_leaf': [1,2,3,4],
    'classification__bootstrap': [True],
}

grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Make predictions
y_pred = grid_search.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))



/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'classification__bootstrap': True, 'classification__max_depth': None, 'classification__min_samples_leaf': 2, 'classification__min_samples_split': 6, 'classification__n_estimators': 100, 'feature_selection__n_features_to_select': 10}
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.50      0.14      0.22         7
           5       0.70      0.86      0.77       100
           6       0.74      0.68      0.71        98
           7       0.83      0.58      0.68        26
           8       1.00      0.50      0.67         2

    accuracy                           0.73       234
   macro avg       0.63      0.46      0.51       234
weighted avg       0.73      0.73      0.72       234



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [184]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
from sklearn.utils import resample
from scipy.stats import zscore

# Loading redwine quality data
df = pd.read_csv('winequality-red.csv', header = 0, delimiter=';')

# Splitting data by class
df_3 = df[df.quality ==3]
df_4 = df[df.quality ==4]
df_5 = df[df.quality ==5]
df_6 = df[df.quality ==6]
df_7 = df[df.quality ==7]
df_8 = df[df.quality ==8]

def remove_outliers(df):
    # Calculates mean and standard deviation of the input df
    mean = df.mean()
    std = df.std()

    # creates a new dataframe for rows that are within their corresponding standard deviation
    new_df = df[(np.abs(df - mean) <= 3*std).all(axis=1)]

    # Return the dataframe with outliers removed
    return new_df

# Applying the function to each dataframe
df_3_new = remove_outliers(df_3)
df_4_new = remove_outliers(df_4)
df_5_new = remove_outliers(df_5)
df_6_new = remove_outliers(df_6)
df_7_new = remove_outliers(df_7)
df_8_new = remove_outliers(df_8)

# Concatenating all new dataframes by row
df_new = pd.concat([df_3_new,df_4_new,df_5_new,df_6_new,df_7_new,df_8_new],axis=0)

# Prepare the data
X = df_new.drop('quality', axis=1)
y = df_new['quality']

# Spltting the data set 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [187]:
df_new.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000
mean,8.304821,0.525475,0.264070,2.388464,0.081448,15.154270,44.151515,0.996711,3.316756,0.641839,10.414153,5.639807
std,1.641415,0.172812,0.190776,0.869068,0.020704,9.323774,30.121700,0.001735,0.141002,0.129084,1.016209,0.814505
min,4.900000,0.120000,0.000000,1.200000,0.012000,1.000000,6.000000,0.990640,2.880000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070750,7.000000,21.000000,0.995600,3.220000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,36.000000,0.996700,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.635000,0.420000,2.600000,0.089000,21.000000,59.000000,0.997800,3.400000,0.720000,11.075000,6.000000
max,13.700000,1.580000,0.760000,6.700000,0.267000,48.000000,155.000000,1.002600,3.780000,1.130000,14.000000,8.000000


In [188]:
from sklearn.metrics import classification_report

# Defining a pipeline with standard scaling, RFE feature selection using RFC, and RFC classification
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42,class_weight='balanced'))),
    ('classification', RandomForestClassifier(random_state=42,class_weight='balanced'))
])

# Defining a dictionary parameter grid to test out for the feature selection and classification parameters
param_grid = {
    'feature_selection__n_features_to_select': [5,6,7,8,9,10,11],
    'classification__n_estimators': [98,99,100,101,102],
    'classification__max_depth': [None],
    'classification__min_samples_split': [1,2,3,4,5,6],
    'classification__min_samples_leaf': [1,2,3,4],
    'classification__bootstrap': [True],
}

# Defining the grid search with the pipeline, parameter grid, and weighted scoring
grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtaining best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Obtaining prediction results
y_pred = grid_search.predict(X_test)

# Printing the classification report
print(classification_report(y_test, y_pred))

Best parameters: {'classification__bootstrap': True, 'classification__max_depth': None, 'classification__min_samples_leaf': 1, 'classification__min_samples_split': 3, 'classification__n_estimators': 99, 'feature_selection__n_features_to_select': 10}
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        10
           5       0.72      0.80      0.76       123
           6       0.70      0.73      0.72       116
           7       0.74      0.64      0.69        36
           8       0.00      0.00      0.00         4

    accuracy                           0.71       291
   macro avg       0.36      0.36      0.36       291
weighted avg       0.68      0.71      0.69       291



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [189]:
from sklearn.metrics import classification_report

# Defining a pipeline with standard scaling, RFE feature selection using RFC, and RFC classification
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42,class_weight='balanced'))),
    ('classification', RandomForestClassifier(random_state=42,class_weight='balanced'))
])

# Defining a dictionary parameter grid to test out for the feature selection and classification parameters. This is altered from above to see what happenes when we remove the option of all features
param_grid = {
    'feature_selection__n_features_to_select': [7,8,9,10],
    'classification__n_estimators': [98,99,100,101,102],
    'classification__max_depth': [None],
    'classification__min_samples_split': [4,5,6],
    'classification__min_samples_leaf': [1,2,3,4],
    'classification__bootstrap': [True],
}

grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Make predictions
y_pred = grid_search.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))


Best parameters: {'classification__bootstrap': True, 'classification__max_depth': None, 'classification__min_samples_leaf': 1, 'classification__min_samples_split': 4, 'classification__n_estimators': 101, 'feature_selection__n_features_to_select': 8}
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.50      0.10      0.17        10
           5       0.75      0.79      0.77       123
           6       0.71      0.76      0.73       116
           7       0.68      0.64      0.66        36
           8       0.00      0.00      0.00         4

    accuracy                           0.72       291
   macro avg       0.44      0.38      0.39       291
weighted avg       0.70      0.72      0.70       291



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
